In [95]:
import numpy as np
import pandas as pd
import os
from urllib.parse import quote, unquote
import pickle

In [96]:
path = '/home/ctang/ttmp/MMCSR/GiantMIDI'
file_path = path + '/full_music_pieces_youtube_similarity_pianosoloprob_split.csv'
GiantMidi = pd.read_csv(file_path)
bootleg_score_path = '/mnt/data0/Datasets/imslp_bootleg_scores'

In [97]:
GiantMidi = GiantMidi[GiantMidi['giant_midi_piano'] == 1.0]
GiantMidi['composer'] = GiantMidi['surname'] + ', ' + GiantMidi['firstname']
GiantMidi = GiantMidi[['composer', 'music', 'audio_name']]
GiantMidi['midi_path'] = ''
GiantMidi['midi_path'] = '/home/ctang/ttmp/MMCSR/GiantMIDI/midis/' + GiantMidi['audio_name'] + '.mid'
GiantMidi.drop(columns=['audio_name'], inplace=True)
GiantMidi['metadata'] = False
GiantMidi['metadata_composer'] = ''
GiantMidi['metadata_title'] = ''
GiantMidi['imslp_url'] = ''
GiantMidi['bootleg_score_path'] = np.nan
GiantMidi.head()

,composer,music,midi_path,metadata,metadata_composer,metadata_title,imslp_url,bootleg_score_path
0,"A., Jag",Je t'aime Juliette,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/A., Jag...",False,,,,NaN
3,"Aaron, Michael",Piano Course,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Aaron, ...",False,,,,NaN
84,"Abel, Frederic",Lola Polka,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Abel, F...",False,,,,NaN
140,"Abramowitsch, A.",3 Nocturnes,/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Abramow...,False,,,,NaN
161,"Abreu, Zequinha",Tico-Tico no fubá,"/home/ctang/ttmp/MMCSR/GiantMIDI/midis/Abreu, ...",False,,,,NaN


In [98]:
with open('/home/ctang/ttmp/MMCSR/pretraining_metadata/all_metadata.pkl', 'rb') as f:
    all_metadata = pickle.load(f)

In [100]:
for row in GiantMidi.itertuples():
    title = row.music
    title1 = title
    midi_path = row.midi_path
    composer = unquote(row.composer)
    if ',' in title and any(char.isdigit() for char in title):
        title1 = ",".join(title.split(",")[:-1])
    title, title1 = unquote(title), unquote(title1)
    composer_lower = composer.lower()
    title_lower = title.lower()
    title1_lower = title1.lower()
    
    bootleg_score_path_1 = os.path.join(bootleg_score_path, composer, title).replace(' ', '_') + '_'
    if os.path.exists(bootleg_score_path_1):
        GiantMidi.at[row.Index, 'metadata'] = True
        GiantMidi.at[row.Index, 'metadata_composer'] = composer
        GiantMidi.at[row.Index, 'metadata_title'] = title
        GiantMidi.at[row.Index, 'bootleg_score_path'] = bootleg_score_path_1

    if composer_lower in (comp.lower() for comp in all_metadata):
        composer_actual = next(comp for comp in all_metadata if comp.lower() == composer_lower)
        titles = all_metadata[composer_actual]
        
        if (title_lower in (t.lower() for t in titles) or 
            any(title1_lower in t.lower() for t in titles) or
            any(t.lower() in title_lower for t in titles)):
            
            GiantMidi.at[row.Index, 'metadata'] = True
            GiantMidi.at[row.Index, 'metadata_composer'] = composer
            
            if title_lower in (t.lower() for t in titles):
                GiantMidi.at[row.Index, 'metadata_title'] = title
            else:
                matching_title = next(t for t in titles if title1_lower in t.lower() or t.lower() in title_lower)
                GiantMidi.at[row.Index, 'metadata_title'] = matching_title
               
            try: 
                GiantMidi.at[row.Index, 'imslp_url'] = all_metadata[composer_actual][GiantMidi.at[row.Index, 'metadata_title']]['url']
                
                my_bootleg_score_path = os.path.join(bootleg_score_path, composer, GiantMidi.at[row.Index, 'metadata_title'])
                # replace spaces with underscores
                my_bootleg_score_path = my_bootleg_score_path.replace(' ', '_') + '_'
                # check if the bootleg score exists
                if os.path.exists(bootleg_score_path):
                    GiantMidi.at[row.Index, 'bootleg_score_path'] = my_bootleg_score_path
                    
            except:
                pass

In [101]:
GiantMidi['bootleg_score_path'].notnull().sum()

7321

In [102]:
# if midi_path is not nan, set has_midi to be true
GiantMidi['has_midi'] = GiantMidi['midi_path'].notnull()

In [103]:
GiantMidi.drop_duplicates(subset=['composer', 'music'], inplace=True)
GiantMidi.groupby(['has_midi', 'metadata']).size()

has_midi  metadata
True      False       3435
          True        7413
dtype: int64

In [104]:
Midi = GiantMidi[GiantMidi['has_midi']]
Midi = Midi.reset_index(drop=True)
Check = Midi[~Midi['metadata']].reset_index(drop=True)

In [107]:
# remove my personal information from midi_path
Midi['midi_path'] = Midi['midi_path'].apply(lambda x: x.replace('/home/ctang/ttmp/MMCSR/', '../'))

In [114]:
Midi.drop(columns=['has_midi', 'imslp_url'], inplace=True)
# rename music to be title
Midi.rename(columns={'music': 'title'}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10848 entries, 0 to 10847
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   composer            10848 non-null  object
 1   title               10848 non-null  object
 2   midi_path           10848 non-null  object
 3   metadata            10848 non-null  bool  
 4   metadata_composer   10848 non-null  object
 5   metadata_title      10848 non-null  object
 6   bootleg_score_path  7321 non-null   object
dtypes: bool(1), object(6)
memory usage: 519.2+ KB


In [115]:
# save midi to disk
Midi.to_csv('/home/ctang/ttmp/MMCSR/GiantMIDI/GiantMidi_match.csv', index=False)

Reasons for not being included:

Among things I inspected, a quarter are not in IMSLP, the others aren't included because:

1. organ pieces and 4-hand piano pieces are included in solo piano, as are some random choral and orchestral pieces; (half)
2. some piece names are in other languages; (1/10)
4. the metadata file doesn't contain the piece - basically no mozart, haydn, liszt, schubert pieces were included (need to look into why) (2/5)
        [update: the solo piano metadata file is missing ~900 pieces]
5. giantmidi has spelling errors (a few)

---

Fixed issues:

1. case sensitive (fixed)
2. contains extra opus numbers (fixed)